# MOVIE DATASET CREATION


In [1]:
# all imports here
from bs4 import BeautifulSoup as bs
import requests

import json

### TASK 1: Get info Box from wikipedia(for one movie) - store in a python dictionary

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

# convert to a beautiful soup object
soup = bs(r.content)

contents = soup.prettify()
# print(contents)

In [3]:
info_box = soup.find(class_="infobox vevent")
# print(info_box.prettify())

info_rows = info_box.find_all("tr")
for row in info_rows:
    print(row.prettify())

<tr>
 <th class="summary" colspan="2" style="text-align:center;font-size:125%;font-weight:bold;font-size:110%;font-style:italic;">
  Toy Story 3
 </th>
</tr>

<tr>
 <td colspan="2" style="text-align:center">
  <a class="image" href="/wiki/File:Toy_Story_3_poster.jpg" title="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3.">
   <img alt="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3." class="thumbborder" data-file-height="326" data-file-width="220" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/6/69/Toy_Story_3_poster.jpg" width="220"/>
  </a>
  <div style="font-size:95%;padding:0.35em 0.35em 0.25em;line-height:1.25em;">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th scope="row" style="white-space:nowra

In [4]:
# (" ", strip=True) joines results with a space and removes extra spaces

def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True)

movie_info = {}

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find("th").get_text(" ", strip=True)
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip=True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value

# print(movie_info)
# movie_info



### TASK 2: Get all info box for all movies

In [5]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

# convert to a beautiful soup object
soup = bs(r.content)

contents = soup.prettify()
# print(contents)

In [24]:
movies = soup.select(".wikitable.sortable i")

def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
         return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip=True)
# movies[0].a['href']
# movies

def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()

def get_info_box(url):
    r = requests.get(url)
    soup = bs(r.content)
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")

    clean_tags(soup)
    movie_info = {}

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text(" ", strip=True)
        elif index == 1:
            continue
        else:
            content_key = row.find("th").get_text(" ", strip=True)
            content_value = get_content_value(row.find("td"))
            movie_info[content_key] = content_value

    return movie_info


In [25]:
get_info_box("https://en.wikipedia.org/wiki/The_Great_Locomotive_Chase")

{'title': 'The Great Locomotive Chase',
 'Directed by': 'Francis D. Lyon',
 'Produced by': ['Lawrence Edward Watkin', 'Walt Disney'],
 'Written by': 'Lawrence Edward Watkin',
 'Starring': ['Fess Parker',
  'Jeffrey Hunter',
  'John Lupton',
  'Jeff York',
  'Slim Pickens'],
 'Music by': 'Paul J. Smith',
 'Cinematography': 'Charles Boyle',
 'Edited by': 'Ellsworth Hoagland',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': 'June 8, 1956',
 'Running time': '85 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Box office': '$1.7 million (US)'}

In [7]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
# convert to a beautiful soup object
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")
# print(len(movies))

base_path = "https://en.wikipedia.org/"
movie_info_list = []

for index, movie in enumerate(movies):
    # if index == 10:
    #     break
    try:
        full_path = base_path + movie['href']
        title = movie['title']

        # print(relative_path, title, sep="\n")
        # print()
        movie_info_list.append(get_info_box(full_path))
    except Exception as e:
        print(movie.get_text())
        print(e)




Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
One Little Indian
'NoneType' object has no attribute 'get_text'
True-Life Adventures
'NoneType' object has no attribute 'find_all'
Spirited Away
'NoneType' object has no attribute 'get_text'
Howl's Moving Castle
'NoneType' object has no attribute 'get_text'
The Nightmare Before Christmas 3D
'NoneType' object has no attribute 'get_text'
Ponyo
'NoneType' object has no attribute 'get_text'
Tales from Earthsea
'NoneType' object has no attribute 'get_text'
The Secret World of Arrietty
'NoneType' object has no attribute 'get_text'
Khoobsurat
'NoneType' object has no attribute 'get_text'
Tini: The Movie
'NoneType' object has no attribute 'get_text'
Born in China
'NoneType' object has no attribute 'get_text'


In [10]:
# print(len(movie_info_list))

# save the dictionary data as a json
def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

save_data('disney_data.json', movie_info_list)

### TASK 3: Clean Our Data
##### 1. Convert values to appropriate datatypes
        - Running Time, Price(Budget+Box Office), date 
##### 2. Remove references
##### 3. Split long list to actual python list
##### 4. Remove extra date as it is not uniform accross all entries



In [18]:
# clean up references ([1], [2], [3], ...)
# remove extra date
# sol: remove by tag, tags containing the ref

# def clean_tags(soup):
#     for tag in soup.find_all(["sup", "span"]):
#         tag.decompose()


In [ ]:
# split long list to ectual list
# sol:

# elif row_data.find("br"):
#          return [text for text in row_data.stripped_strings]